# Westeros Tutorial - Trade in multi-node energy systems

In this tutorial, we demonstrate how a multinode energy system can be built using the *MESSAGEix* framework. The aim of this tutorial is to show:

- how new nodes can be added to a model and how data can be populated for those new nodes
- how a trade hub can be represented
- how bilateral trade can be modeled
- the role of the `message_ix` index set "mode" in representing trade links


**Pre-requisites for running this tutorial**
- You have the *MESSAGEix* framework installed and working.
- You have run Westeros baseline scenario (``westeros_baseline.ipynb``) and solved it successfully.

This tutorial was presented by [Behnam Zakeri](https://iiasa.ac.at/staff/behnam-zakeri) at the **MESSAGEix Community Meeting** May 2022. Please feel free to suggest improvements through issues and pull-requests.

In [2]:
import ixmp

# Loading the modeling platform
mp = ixmp.Platform()

## 1. Adding new nodes and populating input data for them

First, we clone the existing scenario "baseline" as the basis for our multinode model. Next, we add two model regions called "Essos" and "Stepstones" as two neighboring regions of Westeros (for more details on the geography of the system we are discussing, please refer to [this Wiki](https://gameofthrones.fandom.com/wiki/Westeros)). Then, we solve the model for this three-node system and check the outcome. 

In [3]:
from message_ix import Scenario

# Loading baseline scenario
model = "Westeros Electrified"
scenario = "baseline"
base = Scenario(mp, model, scenario)

This Scenario has a solution, use `Scenario.remove_solution()` or `Scenario.clone(..., keep_solution=False)`


Let's make sure our "baseline" scenario has a solution. We will need this solution later, as we will compare the output of different scenarios. If your "baseline" does not have a solution, i.e., you get an output of `False` from the cell below, please run ``westeros_baseline.ipynb`` and solve it again.

In [4]:
# Check if the "baseline" scenario has a solution
base.has_solution()

True

In [5]:
# Cloning "baseline" to a new scenario for our work
new_scenario = "multinode"
scen = base.clone(scenario=new_scenario, keep_solution=False)

# Checking out the scenario for editing
scen.check_out()

### 1.1. Adding new nodes
Adding new nodes to a *MESSAGEix* scenario is like adding elements to any other sets, i.e., by using the method `add_set()`. The framework does not assign any relationship between nodes being added though `add_set()`. This means no "parent" node or hirerarchy can be defined.

In [6]:
# Adding new nodes to the set "node"
nodes_new = ["Essos", "Stepstones"]
scen.add_set("node", nodes_new)

In [7]:
# Let's check the set of "node"
scen.set("node")

0         World
1      Westeros
2         Essos
3    Stepstones
dtype: object

### 1.2. Populating data for the new nodes
By now, the model has no data related to these new nodes, e.g., no technology parameter is defined for them. Here, we can copy the data of "Westeros" to the new nodes. This way, we do not have to repeat all the steps for adding input data as we did for "Westeros" in ``westeros_baseline.ipynb`` for each new node. We can simply use `message_ix.rename()` feature and keep the original node ("Westeros") by using `keep=True`.

In [8]:
# Copying data from "Westeros" to the new nodes
for node in nodes_new:
    scen.rename("node", {"Westeros": node}, keep=True)

Let's check and see if the new nodes are correctly added to parameters, e.g., in parameter `demand` in one of the model years.

In [9]:
scen.par("demand", {"year": 700})

,node,commodity,level,year,time,value,unit
0,Westeros,light,useful,700,year,55.0,GWa
1,Essos,light,useful,700,year,55.0,GWa
2,Stepstones,light,useful,700,year,55.0,GWa


As we can see from the DataFrame above, the data has been populated for the new nodes, with equal values to "Westeros". Now, let's commit changes and solve the scenario and see what happens...

In [10]:
# Committing and solving
scen.commit("new nodes added")
scen.solve()

Right! the scenario solved (hopefully for you too). As the input data were the same for the three nodes, we expect output results will be the same too. Let's check some output like "activity" (`ACT`) for a technology in a specific year:

In [11]:
scen.var("ACT", {"technology": "wind_ppl", "year_act": 700})

,node_loc,technology,year_vtg,year_act,mode,time,lvl,mrg
0,Westeros,wind_ppl,690,700,standard,year,12.176560,0.0
1,Westeros,wind_ppl,700,700,standard,year,1.560259,0.0
2,Essos,wind_ppl,690,700,standard,year,12.176560,0.0
3,Essos,wind_ppl,700,700,standard,year,1.560259,0.0
4,Stepstones,wind_ppl,690,700,standard,year,12.176560,0.0
5,Stepstones,wind_ppl,700,700,standard,year,1.560259,0.0


You should see equal results for all nodes. Now, we have a three-node model, but as there is no link between the three nodes, and their input data is exactly the same, the output results are also the same. In the next section, we use this model to add trade links.

## 2. Representing  a trade hub in a multi-node model
In this part, we model a trade hub. A trade hub is like a pool that each node can use to export energy commodities to, or import from. Examples can be a central electricity grid in a country, to which many cities and power plants are connected. Or this can be an energy market where bilateral trade is not important to track, but only buyers and sellers are relevant. The schematic below shows possible configurations of a trade hub. 
In a trade hub:
- import to and export from the hub in each node may have capacity (size)
- there is no bilateral trade between nodes
- the hub may have capacity (size)

<img src='_static/hub.JPG'>

In [12]:
# We clone a new scenario and call it "scen2"
scen2 = scen.clone(scenario="multinode_hub", keep_solution=False)
scen2.check_out()

### 2.1. Adding information related to the trade hub
We need to specify at a minimum the following information in the model for a simple trade hub:
- a "node" name for the location of the trade hub: we add a new node called "hub"
- a "level" name for importing/exporting energy commodity: we add a new level called "trade"
- import and export technologies in each node

Please note that we would need to add more detail, if we wanted to represent the size of the hub, i.e.:
- a "level" name from which energy commodity is imported
- a "level" name to which energy commodity is exported 
- a "technology" representing the trade size in the hub

But for the sake of simplicity, we skip the trade size (capacity) parameters here.

In [13]:
# Adding new set elements for the hub
scen2.add_set("node", "hub")
scen2.add_set("level", "trade")

# Adding new electricity import and export technologies
scen2.add_set("technology", ["elec_imp", "elec_exp"])

### 2.2. Linking import/export technologies to the hub
Representing a network or trade link in *MESSAGEix* can be done through `input` and `output` parameters. This means there is no specific parameter related to trade. Both `input` and `output` parameters are defined with two indexes related to the location.

For example, `input` of a technology can be defined with respect to where that technology is located (`node_loc`), and where the input commodity of that technology originates from (`node_origin`). For the `output` parameter, the destination of the output commodity can be defined by `node_dest`. You can check these index names by running the `Scenario.idx_names()` method (see below).

For linking electricity import and export technologies to the hub, we use the level of "secondary" in the Westeros model, which is the point before the transmission and distribution (T&D) grid in each node.

In [14]:
# Checking index names of a parameter (useful when adding data to that parameter)
scen2.idx_names("input")

['node_loc',
 'technology',
 'year_vtg',
 'year_act',
 'mode',
 'node_origin',
 'commodity',
 'level',
 'time',
 'time_origin']

In [15]:
scen2.set("commodity")

0    electricity
1          light
dtype: object

#### 2.2.1. Parametrization of `input` and `output` for import technologies
Here, we show the general parametrization of 'input' and 'output'. We will add a loss due to efficiency of the system in the next step.

In [16]:
from message_ix import make_df

# Parametrization of "input" for import technologies
# The origin of import is the level of "trade" from hub
model_years = list(scen2.set("year"))
base_input = {
    "node_origin": "hub",
    "technology": "elec_imp",
    "commodity": "electricity",
    "level": "trade",
    "year_vtg": model_years,
    "year_act": model_years,
    "mode": "standard",
    "time": "year",
    "time_origin": "year",
    "value": 1,
    "unit": "-",
}

# We add this data for each node (other than "hub")
for country in ["Westeros", "Essos", "Stepstones"]:
    inp = make_df("input", **base_input, node_loc=country)
    scen2.add_par("input", inp)

In [17]:
# Parametrization of "output" for import technologies
# The destination of import is the level of "secondary" in each country
base_output = {
    "technology": "elec_imp",
    "commodity": "electricity",
    "level": "secondary",
    "year_vtg": model_years,
    "year_act": model_years,
    "mode": "standard",
    "time": "year",
    "time_dest": "year",
    "value": 1,
    "unit": "-",
}

# We add this data for each node (other than "hub")
for country in ["Westeros", "Essos", "Stepstones"]:
    out = make_df("output", **base_output, node_loc=country, node_dest=country)
    scen2.add_par("output", out)

#### 2.2.2. Adding loss due to efficiency of the system
We assume a 5% loss of electricity when trading in the hub for each node. This can be represented as 1.05 input value for each 1 output unit of electricity.

In [18]:
# The origin of export is the level of "secondary" in each country
for country in ["Westeros", "Essos", "Stepstones"]:
    base_input.update(
        dict(technology="elec_exp", node_origin=country, level="secondary", value=1.05)
    )
    inp = make_df("input", **base_input, node_loc=country)
    scen2.add_par("input", inp)

In [19]:
# The destination of export is the level of "trade" in the hub
for country in ["Westeros", "Essos", "Stepstones"]:
    base_output.update(dict(technology="elec_exp", node_dest="hub", level="trade"))
    out = make_df("output", **base_output, node_loc=country)
    scen2.add_par("output", out)

### 2.3. Modifying `demand` and solving
We change the demand for "light", as such for electricity, in the nodes "Essos" and "Stepstones", by +15% and -50% relative to "Westeros", repsectively. This is for illustration purposes, but can be based on population, daylight time, etc. in these nodes. 

In [20]:
# Changing "demand" in Essos and Stepstones
change = {"Essos": +0.15, "Stepstones": -0.5}
for node, value in change.items():
    # Loading data of "demand"
    df = scen2.par("demand", {"node": node})
    # Multiplying by the value
    df["value"] *= 1 + value
    # Adding new demand to the scenario
    scen2.add_par("demand", df)

In [21]:
# Committing and solving
scen2.commit("new nodes added")
scen2.solve()
scen2.set_as_default()

#### Checking the results
Let's check the output results to see if any trade has occured or not:

In [22]:
scen2.var("ACT", {"technology": ["elec_imp", "elec_exp"], "year_act": 700})

,node_loc,technology,year_vtg,year_act,mode,time,lvl,mrg
0,Westeros,elec_imp,700,700,standard,year,1.560259,0.000000
1,Westeros,elec_exp,700,700,standard,year,0.000000,36.794377
2,Essos,elec_imp,700,700,standard,year,10.726926,0.000000
3,Essos,elec_exp,700,700,standard,year,0.000000,36.794377
4,Stepstones,elec_imp,700,700,standard,year,0.000000,35.042264
5,Stepstones,elec_exp,700,700,standard,year,12.287186,0.000000


### 2.4. Role of trade
Looking at the results of activity (`ACT`), we can see that the electricity trade is balanced, i.e., the sum of imports equals the sume of exports in each year. "Stepstones" is the exporter as demand for lighting is the lowest in this node and extra electricity can be exported to the other two nodes with higher demand.

> **Note:** In this example, we represented trade simply with activity. In more realistic examples, trade links can be modeled with capacity related parameters, i.e., lifetime, investment, capacity factor, etc.

#### Questions:
- Compare the price of electricity in "Westeros" before and after the trade? What is the role of trade? (Tip: you can use variable `PRICE_COMMODITY` to compare the prices between "baseline" and trade scenarios)

#### Solution:

In [23]:
base.var("PRICE_COMMODITY", {"commodity": "light"})

,node,commodity,level,year,time,lvl,mrg
0,Westeros,light,useful,700,year,511.028293,0.0
1,Westeros,light,useful,710,year,162.039539,0.0
2,Westeros,light,useful,720,year,161.002627,0.0


In [24]:
scen2.var("PRICE_COMMODITY", {"commodity": "light"})

,node,commodity,level,year,time,lvl,mrg
0,Westeros,light,useful,700,year,166.445336,0.0
1,Westeros,light,useful,710,year,162.039539,0.0
2,Westeros,light,useful,720,year,161.002627,0.0
3,Essos,light,useful,700,year,166.445336,0.0
4,Essos,light,useful,710,year,162.039539,0.0
5,Essos,light,useful,720,year,161.002627,0.0
6,Stepstones,light,useful,700,year,161.402965,0.0
7,Stepstones,light,useful,710,year,162.039539,0.0
8,Stepstones,light,useful,720,year,161.002627,0.0


## 3. Bilateral trade
In this part, we model bilateral trade links, i.e., direct trade between two nodes. A bilateral trade representation is useful when it is important to link importer and exporter nodes via an infrastructure that cannot be shared with other nodes. Bilateral trade in a multi-node model can form a "meshed" network of capacitated lines. Therefore, nodes can trade commodities, if an economic trade route subject to possible losses exists between exporting and importing nodes.

For the bilateral trade, again, we use `input` and `output` parameters to link trading nodes directly. Bilateral trade can exist in one direction (one-way), such as the flow of water from one node to another, or bi-directional (two-way), such as an electricity transmission line.

<img src='_static/bilateral.JPG'>

In [25]:
# We clone a new scenario and call it "scen3"
scen3 = scen2.clone(scenario="multinode_bilateral", keep_solution=False)
scen3.check_out()

In [26]:
# We remove previous trade links
for tec in ["elec_exp", "elec_imp"]:
    scen3.remove_set("technology", tec)

### 3.1. Bilateral trade and "mode" of operation
We can represent bilateral trade similar to the Part (2) above, i.e., with two distinct import and export technologies. Alternatively, we can only use one technology for the trade link, but with two "mode"s of operation, i.e., import and export.

Using "mode" of operation is useful when the import and export links are a single technology, like electricity interconnectors. When using "mode", the "capacity" related parameters are the same for the technology, but "activity" related parameters should be defined per "mode".

We need to specify the following information in the model for a direct bilateral bi-dimensional trade link:
- a "node" name for the location of the trade link: we use the "hub" node.
- a single trade technology: we add "elec_trade"
- two "mode"s of operation for the trade link.

In [27]:
# Adding new electricity trade technology
scen3.add_set("technology", ["elec_trade"])

### 3.2. Linking two nodes
For linking two nodes through a single electricity trade technology, we should decide where the location of this technology is. The trade link can be located in either of the nodes, and so, will have the investment and costs related to that node.

In this case, we choose the node "hub" as the location of the trade link between two nodes of "Westeros" and "Stepstones". However, the user can decide to set the location of a trade link in the node of origin or destination based on their case study. In this example, this does not change anything in the results. But if the trade link had `inv_cost`, a node that is the location of the trade link would bear that investment.

We directly link the two nodes at the level of "secondary", which is the point before the transmission and distribution (T&D) grid in each node.

In [28]:
# Adding modes for the bilateral trade
# We add "W-to-S" for Westeros --> Stepstone; and "S-to-W" for the opposite direction
scen3.add_set("mode", ["W-to-S", "S-to-W"])

#### 3.2.1. Parametrization of `input` and `output` for the bilateral two-way trade link
Similar to the Part 2, we parametrize `input` and `output` parameters for the trade link. We assume a 5% loss of electricity in each direction. This can be represented as 1.05 input value for each 1 output unit of electricity.

In [29]:
# Parametrization of "input" for the trade link in the first direction
# We assign the "mode" of "W-to-S" for the direction of trade
# from Westeros --> Stepstones
base_input = {
    "node_origin": "Westeros",
    "technology": "elec_trade",
    "commodity": "electricity",
    "level": "secondary",
    "year_vtg": model_years,
    "year_act": model_years,
    "mode": "W-to-S",
    "time": "year",
    "time_origin": "year",
    "value": 1.05,
    "unit": "-",
}

# We add this data to the model
inp = make_df("input", **base_input, node_loc="hub")
scen3.add_par("input", inp)

In [30]:
# Parametrization of "output" for the trade link in the first direction
# We assign the "mode" of "W-to-S" for the direction of trade
# from Westeros --> Stepstones
base_output = {
    "node_dest": "Stepstones",
    "technology": "elec_trade",
    "commodity": "electricity",
    "level": "secondary",
    "year_vtg": model_years,
    "year_act": model_years,
    "mode": "W-to-S",
    "time": "year",
    "time_dest": "year",
    "value": 1,
    "unit": "-",
}

# We add this data to the scenario
out = make_df("output", **base_output, node_loc="hub")
scen3.add_par("output", out)

In [31]:
# Parametrization of "input" for the trade link in the second direction
# We assign the "mode" of "S-to-W" for the direction of trade
# from Stepstones --> Westeros
base_input.update(dict(node_origin="Stepstones", mode="S-to-W"))
inp = make_df("input", **base_input, node_loc="hub")
scen3.add_par("input", inp)

In [32]:
# Parametrization of "output" for the trade link in the second direction
# We assign the "mode" of "S-to-W" for the direction of trade
# from Stepstones --> Westeros
base_output.update(dict(node_dest="Westeros", mode="S-to-W"))
out = make_df("output", **base_output, node_loc="hub")
scen3.add_par("output", out)

#### 3.2.2. Parametrization of bilateral trade but only in one direction
For illustration, we make an electricity transmission link between nodes "Westeros" and "Essos", but only in one direction, from "Westeros". The difference here is that we do not need to add a second "mode" of operation to the trade link. We use a new technology "elec_export" for this trade link. We assume a 5% loss of electricity in each direction. This can be represented as 1.05 input value for each 1 output unit of electricity.

In [33]:
# Parametrization of "input" for the one-way trade link
# Origin is Westeros
# We assign the "mode" of "standard" as there is only one direction
scen3.add_set("technology", "elec_export")
base_input.update(
    dict(node_origin="Westeros", technology="elec_export", mode="standard")
)
inp = make_df("input", **base_input, node_loc="hub")
scen3.add_par("input", inp)

In [34]:
# Parametrization of "output" for the one-way trade link
# Destination is Essos
base_output.update(dict(node_dest="Essos", technology="elec_export", mode="standard"))
out = make_df("output", **base_output, node_loc="hub")
scen3.add_par("output", out)

In [35]:
# Committing and solving
scen3.commit("new nodes added")
scen3.solve()

Let's check the output results to see if any trade has occured between the nodes or not:

In [36]:
scen3.var("ACT", {"technology": ["elec_trade", "elec_export"], "year_act": 700})

,node_loc,technology,year_vtg,year_act,mode,time,lvl,mrg
0,hub,elec_trade,700,700,W-to-S,year,0.000000,71.83664
1,hub,elec_trade,700,700,S-to-W,year,12.823532,0.00000
2,hub,elec_export,700,700,standard,year,10.726926,0.00000


###  3.3. Role of trade
The results of activity (`ACT`) show that the electricity trade is in the mode of "S-to-W", i.e., from Stepstones to Westeros, in the bi-directional line. Stepstones is the exporter as demand for lighting is the lowest in this node, while the potential supply technologies are the same. Hence, the relatively cheaper electricity can be exported to Westeros with higher demand. Further, Westeros exports to Essos, as the latter has the highest demand for electricity. Here, Westeros is acting as an intermediate node in the trade network to pass electricity from Stepstones to Essos, of course with additional losses. Because there is no trade possibilities between Stepstones and Essos directly.

#### Questions:
- Why is the price of "light" higher in Essos than Westeros in year 700? (Compare this with Part 2)

In [37]:
scen3.var("PRICE_COMMODITY", {"commodity": "light", "year": 700})

,node,commodity,level,year,time,lvl,mrg
0,Westeros,light,useful,700,year,166.445336,0.0
1,Essos,light,useful,700,year,171.739825,0.0
2,Stepstones,light,useful,700,year,161.402965,0.0


### Exercise: Trade with an external market/system
We demonstrated the possibilites for a hub-based and bilateral trade representation in MESSAGEix. In some case studies, a trade link needs to be facilitated with an external market or system, e.g., to neighbouring nodes or to the world outside of the boundary of the modeled nodes. This is very common, as it is not possible or desirable to model all neighbouring regions or the external system, and a model needs to be limited to a certain boundary. Hence, each node individually and/or the hub can be linked to an external market (e.g., to global commodity markets represented at "World").

<img src='_static\external.JPG'>

In this exercise, you should establish trade with an external market in the Westeros multi-node model. This trade link can be through a hub or directly from a node. You can specify the price of electricity ($/kWa) in the external market as follows: {690: 100, 700: 97, 710: 93, 720: 87}.

<details>
  <summary><font size="3" color="darkgreen"><b>Click for hints</b></font></summary>
       
   * We can model this by adding an electricity importing technology that imports electricity to the node "hub". This technology, however, has no `input` parameter (because we do not model the commodity balance outside of the boundary of our model).
    However, we can define `var_cost` for this trade link to represent the price of commodity in the external market.
    
<details>
   <summary><font size="3" color="darkblue"><b>Click for the code</b></font></summary>

```python
# We clone a new scenario from the scenario with a central hub.
scen2 = message_ix.Scenario(mp, model, "multinode_hub")
scen4 = scen2.clone(scenario="multinode_hub_external_trade", keep_solution=False)
scen4.check_out()
# We link the central hub to an external electricity market.
# First, we add a new technology:
scen4.add_set("technology", "elec_external_imp")
# Then, we define parameter "output" for importing electricity to "hub"
# Here, we don't need to define "input", as this is considered outside the boundary
# conditions of our model
base_output = {
    "node_dest": "hub",
    "node_loc": "hub",
    "technology": "elec_external_imp",
    "commodity": "electricity",
    "level": "trade",
    "year_vtg": model_years,
    "year_act": model_years,
    "mode": "standard",
    "time": "year",
    "time_dest": "year",
    "value": 1,
    "unit": "-",
}
# We add this data to the scenario
scen4.add_par("output", make_df("output", **base_output))
# At this stage, imported electricity has no cost. We define variable cost for that as
# asked in the question.
base_output.update({"value": [100, 97, 93, 87], "unit": "USD/kWa"})
scen4.add_par("var_cost", make_df("var_cost", **base_output))
scen4.commit("")
# Finally, we can solve the scenario
scen4.solve()
# Let's check the price of electricity
print(
    "Price of electricity:\n",
    scen4.var("PRICE_COMMODITY", {"commodity": "electricity", "node": "hub"}),
)
# We observe that the price of electricity in years 710 and 720 is set by the cost of
# importing electricity. But in 700 the cost of trading electricity at the internal hub
# is lower than the cost of importing. Hence, if we check the activity of the import
# technology, it should not show any import in 700 (see column "lvl"):
print(
    "Activity of the electricity importing technology:\n",
    scen4.var("ACT", {"technology": "elec_external_imp"}),
)
```

</details>
</details>

In [38]:
# Close the connection to the database at the end
mp.close_db()